In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
dataset=pd.read_csv("Social_Network_Ads.csv")

In [3]:
dataset

,User ID,Gender,Age,EstimatedSalary,Purchased
0,15624510,Male,19,19000,0
1,15810944,Male,35,20000,0
2,15668575,Female,26,43000,0
3,15603246,Female,27,57000,0
4,15804002,Male,19,76000,0
...,...,...,...,...,...
395,15691863,Female,46,41000,1
396,15706071,Male,51,23000,1
397,15654296,Female,50,20000,1
398,15755018,Male,36,33000,0


In [4]:
dataset=pd.get_dummies(dataset,drop_first=True)

In [5]:
dataset

,User ID,Age,EstimatedSalary,Purchased,Gender_Male
0,15624510,19,19000,0,True
1,15810944,35,20000,0,True
2,15668575,26,43000,0,False
3,15603246,27,57000,0,False
4,15804002,19,76000,0,True
...,...,...,...,...,...
395,15691863,46,41000,1,False
396,15706071,51,23000,1,True
397,15654296,50,20000,1,False
398,15755018,36,33000,0,True


In [6]:
dataset=dataset.drop("User ID",axis=1)

In [7]:
dataset["Purchased"].value_counts()

Purchased
0    257
1    143
Name: count, dtype: int64

In [8]:
dataset.columns

Index(['Age', 'EstimatedSalary', 'Purchased', 'Gender_Male'], dtype='object')

In [9]:
independent=dataset[['Age','EstimatedSalary','Gender_Male']]

In [10]:
independent.shape

(400, 3)

In [11]:
dependent=dataset[['Purchased']]

In [12]:
dependent

,Purchased
0,0
1,0
2,0
3,0
4,0
...,...
395,1
396,1
397,1
398,0


In [13]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(independent,dependent,test_size=1/3,random_state=0)

In [14]:
from sklearn.preprocessing import StandardScaler
sc=StandardScaler()
x_train=sc.fit_transform(x_train)
x_test=sc.transform(x_test)

In [15]:
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
param_grid={"kernel":["sigmoid","poly","linear","rbf"],"C":[10,100,1000,2000,3000]}
grid=GridSearchCV(SVC(probability=True),param_grid,refit=True,verbose=3,n_jobs=-1,scoring='f1_weighted')
grid.fit(x_train,y_train)

Fitting 5 folds for each of 20 candidates, totalling 100 fits


C:\Users\saran\anaconda3\Lib\site-packages\sklearn\utils\validation.py:1184: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


GridSearchCV(estimator=SVC(probability=True), n_jobs=-1,
             param_grid={'C': [10, 100, 1000, 2000, 3000],
                         'kernel': ['sigmoid', 'poly', 'linear', 'rbf']},
             scoring='f1_weighted', verbose=3)

In [16]:
re=grid.cv_results_
grid_predictions=grid.predict(x_test)

In [17]:
from sklearn.metrics import confusion_matrix
cm=confusion_matrix(y_test,grid_predictions)

In [18]:
from sklearn.metrics import classification_report
clf=classification_report(y_test,grid_predictions)

In [19]:
print("The Report:\n",clf)

The Report:
               precision    recall  f1-score   support

           0       0.92      0.94      0.93        85
           1       0.89      0.86      0.88        49

    accuracy                           0.91       134
   macro avg       0.91      0.90      0.90       134
weighted avg       0.91      0.91      0.91       134



In [20]:
print("confusion matrix:\n",cm) 

confusion matrix:
 [[80  5]
 [ 7 42]]


In [21]:
from sklearn.metrics import f1_score
f1_macro=f1_score(y_test,grid_predictions,average='weighted')
print("The f1 macro value for best parameter {}:".format(grid.best_params_),f1_macro)

The f1 macro value for best parameter {'C': 100, 'kernel': 'rbf'}: 0.9100355779243318


In [22]:
from sklearn.metrics import roc_auc_score
roc_auc_score(y_test,grid.predict_proba(x_test)[:,1])

0.9539015606242497

In [23]:
table=pd.DataFrame.from_dict(re)

In [24]:
table

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_C,param_kernel,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.034377,2.296432e-02,0.018752,0.006250,10,sigmoid,"{'C': 10, 'kernel': 'sigmoid'}",0.762677,0.738916,0.753180,0.778067,0.766556,0.759879,0.013172,16
1,0.021875,7.654870e-03,0.015626,0.000001,10,poly,"{'C': 10, 'kernel': 'poly'}",0.799620,0.808392,0.787943,0.924528,0.901744,0.844445,0.056918,6
2,0.021876,7.654753e-03,0.021876,0.007654,10,linear,"{'C': 10, 'kernel': 'linear'}",0.776290,0.790949,0.698235,0.923510,0.901744,0.818146,0.083619,11
3,0.015626,8.313940e-07,0.025002,0.007655,10,rbf,"{'C': 10, 'kernel': 'rbf'}",0.867478,0.886792,0.869709,0.944161,0.943041,0.902236,0.034431,2
4,0.018749,6.249881e-03,0.025003,0.007654,100,sigmoid,"{'C': 100, 'kernel': 'sigmoid'}",0.808927,0.714931,0.698113,0.796284,0.766556,0.756962,0.043745,17
5,0.068753,1.875097e-02,0.018752,0.006250,100,poly,"{'C': 100, 'kernel': 'poly'}",0.822092,0.787284,0.766556,0.924528,0.901744,0.840441,0.062370,7
6,0.056252,1.250098e-02,0.018752,0.006250,100,linear,"{'C': 100, 'kernel': 'linear'}",0.776290,0.790949,0.698235,0.923510,0.901744,0.818146,0.083619,11
7,0.021876,7.655181e-03,0.021877,0.007655,100,rbf,"{'C': 100, 'kernel': 'rbf'}",0.867478,0.886792,0.870362,0.944161,0.943041,0.902367,0.034308,1
8,0.015625,7.231594e-07,0.018751,0.006250,1000,sigmoid,"{'C': 1000, 'kernel': 'sigmoid'}",0.808927,0.714931,0.698113,0.796284,0.766556,0.756962,0.043745,17
9,0.525023,1.696239e-01,0.021879,0.012501,1000,poly,"{'C': 1000, 'kernel': 'poly'}",0.822092,0.765553,0.766556,0.924528,0.901744,0.836095,0.066541,8


In [25]:
age_input=int(input("age:"))
Es_salary_input=int(input("Estimated salary:"))
gender_input=int(input("Gender male or female:"))

age:24
Estimated salary:5000
Gender male or female:1


In [26]:
future_predictions=grid.predict([[age_input,Es_salary_input,gender_input]])
print("future predictions:{}".format(future_predictions))

future predictions:[1]
